In [1]:
# importuje niezbędne biblioteki
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [2]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [3]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [4]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [5]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [6]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2019-12-12
Date range to YYYY-MM-DD 2020-12-11


In [7]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [8]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [9]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [10]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [12]:
DF3['Miot ogółem'] = len(DF3[(DF3.LRASA1 == 10)])

In [13]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Kolejny miot','Rasa','Mioty szt.', 
                                   'Mioty %','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia',
                                   'Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.NMIOT == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT
# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DT, new,rasa):
    miot = DT['NMIOT'].unique()
    new['Kolejny miot'] = miot[0]
    new['Mioty szt.'] = len(DT)
    new['Mioty %'] = np.around(len(DT)/len(DF3[(DF3.LRASA1 == rasa)])*100,decimals=2)
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DT['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DT['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DT['ST21'].mean(), decimals=2)
    new['Okres miedzymiotu(dni)'] = np.around(DT['OMM'].mean())

    return new

In [14]:
race_dict = {
    10 : "WBP",
    20 : "PBZ",
    30 : "Złotnicka Biała",
    40 : "Póławska",
    50 : "Złotnicka Pstra",
    60 : "Hampshire",
    70 : "Duroc",
    80 : "Pietrain",
    90 : "Linia 990"}

In [15]:
wbp = createDT('Wbp')
WBP = sorted2DT(DF3,10,1)
all_count(WBP,wbp,10)

,Kolejny miot,Rasa,Mioty szt.,Mioty %,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Okres miedzymiotu(dni)
1,1,Wbp,736,22.2,11.74,10.99,5.96,NaN


In [16]:
completeDF = pd.DataFrame()
for race in race_dict:
    dokladna_rasa = DF3[(DF3.LRASA1 == race)]
    mioty = dokladna_rasa['NMIOT'].unique()
    for i in mioty:
        aDF = createDT(race_dict[race])
        bDF = sorted2DT(DF3,race,i)
        rdy = all_count(bDF,aDF,race)
        completeDF = pd.concat([completeDF,rdy])

In [17]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
completeDF.set_index(['Rasa','Kolejny miot']).sort_values(by = ['Rasa','Kolejny miot'])

Mioty szt.  Mioty %  \
Rasa            Kolejny miot                        
Duroc           1                    103    18.53   
                2                    103    18.53   
                3                     96    17.27   
                4                     50     8.99   
                5                     51     9.17   
                6                     48     8.63   
                7                     36     6.47   
                8                     22     3.96   
                9                     11     1.98   
                10                     8     1.44   
                11                     6     1.08   
                12                     6     1.08   
                13                     7     1.26   
                14                     4     0.72   
                16                     1     0.18   
                17                     2     0.36   
                18                     2     0.36   
Hampshire       1                      5    19.23   
                2                      4    15.38   
                3                      1     3.85   
                4                      1     3.85   
                6                      1     3.85   
                7                      2     7.69   
                10                     1     3.85   
                11                     1     3.85   
                13                     3    11.54   
                14                     4    15.38   
                15                     3    11.54   
Linia 990       1                    335    21.20   
                2                    250    15.82   
                3                    228    14.43   
                4                    197    12.47   
                5                    177    11.20   
                6                    151     9.56   
                7                    106     6.71   
                8                     61     3.86   
                9                     41     2.59   
                10                    19     1.20   
                11                     8     0.51   
                12                     6     0.38   
                13                     1     0.06   
PBZ             1                   1287    25.07   
                2                    999    19.46   
                3                    758    14.76   
                4                    553    10.77   
                5                    407     7.93   
                6                    344     6.70   
                7                    262     5.10   
                8                    200     3.90   
                9                    120     2.34   
                10                    85     1.66   
                11                    56     1.09   
                12                    36     0.70   
                13                    18     0.35   
                14                     6     0.12   
                15                     2     0.04   
                17                     1     0.02   
Pietrain        1                     95    18.20   
                2                     86    16.48   
                3                     77    14.75   
                4                     69    13.22   
                5                     56    10.73   
                6                     44     8.43   
                7                     35     6.70   
                8                     21     4.02   
                9                     13     2.49   
                10                     9     1.72   
                11                     7     1.34   
                12                     8     1.53   
                13                     1     0.19   
                14                     1     0.19   
Póławska        1                    590    17.79   
                2                    505    15.23   
                3                    590    17.79   
  